In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


#!/usr/bin/env python
# coding: utf-8

# In[1]:


import streamlit as st
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import pdfplumber
import pandas as pd
import numpy as np
import io
import base64
import cv2



# In[2]:


pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


# In[3]:


def load_image(image_file):
    img = Image.open(image_file)
    return img

def parse_pdf_file(pdf_file):
    out_put = {}
    out_put["text"] = parse_text_pdf_file(pdf_file)
    df_list = parse_pdfplumber_tables(pdf_file)
    #print(df_list)
    if len(df_list) != 0:
        out_put["length"] = len(df_list)
        out_put["tables"] = df_list
        return out_put
    out_put["length"] = 0
    out_put["tables"] = []
    return out_put


def construct_dataframe(data):
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

def parse_pdfplumber_tables(pdf_file):
    df_list = []
    with pdfplumber.open(pdf_file) as pdf:
        pages = pdf.pages
        for i,pg in enumerate(pages):
            file_page = pdf.pages[i]
            tables = file_page.extract_tables()
            if len(tables) !=0:
                for table in tables:
                    df_list.append(construct_dataframe(table))
    return df_list


def parse_text_pdf_file(pdf_file):
    
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for pdf_page in pdf.pages:
            if pdf_page.extract_text(x_tolerance=3, y_tolerance=3) is not None:
                text = pdf_page.extract_text(x_tolerance=3, y_tolerance=3)
                text += "\n"
            else:
                text = parse_text_pytesseract_file(pdf_file)
    return text
    #return parse_text_pytesseract_file(pdf_file)


def parse_image_file(img):
    text = ""
    scale_factor = 2
    img = load_image(img)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (0,0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
    thres, thresh_img = cv2.threshold(img, 225, 255, cv2.THRESH_BINARY)
    config = "--oem 3 --psm 6"
    text = pytesseract.image_to_string(thresh_img, config=config, lang="eng")
    return text


def parse_text_pytesseract_file(pdf_file):
    pages = convert_from_path(pdf_file,poppler_path=r'C:\Users\mukes\Downloads\poppler-utils\poppler-0.68.0\bin')
    for page in pages:
        text += parse_image_file(page)
        text += "\n"
    return text

       
       
def get_csv_download_link(df_list):
    towrite = io.BytesIO()
    for df in df_list:
       # downloaded_file = df.to_csv(towrite, encoding="utf-8", index=False, header=True)
        downloaded_file = df.to_excel(towrite, encoding='utf-8', index=False, header=True)
    towrite.seek(0)
    b64 = base64.b64encode(towrite.read()).decode()
    return f'<a href="data:file/csv;base64,{b64}" download="gen_output.csv"> Download csv file</a>'

def get_csv_download_link_c(df_list):
    towrite = io.BytesIO()
    df = pd.DataFrame.from_dict(map(dict,df_list))
    csv = df.to_csv(towrite,header=True, index=False,encoding="utf-8")
    towrite.seek(0)
    b64 = base64.b64encode(csv.encode()).decode()  # some strings <-> bytes conversions necessary here
    href = f'<a href="data:file/csv;base64,{b64}" download="gen_output.csv"> Download csv file</a>'
    return href
   

def get_text_download_link(text):
    b64 = base64.b64encode(text.encode()).decode()
    return f'<a href="data:file/txt;base64,{b64}" download="gen_output.txt"> Download text file</a>'


# In[4]:


st.title('Ml Applications')

menu =["OCR App","Q/A System"]

choice = st.sidebar.selectbox("Menu",menu)

if choice == "OCR App":
    st.title("OCR APP")
    input_file = st.file_uploader("Upload File", type = ["png","jpg","jpeg","pdf"])
    if st.button("Process"):
        if input_file is not None:
            
            text = ""
            
            if input_file.type in ["image/png","image/jpg","image/jpeg"]:
                
                text =parse_image_file(input_file)
                
                st.write(text)
            elif input_file.type in ["application/pdf"]:
                
                out_put = parse_pdf_file(input_file)
                
                text = out_put['text']
                #st.write(out_put['tables'][0])
                #df = pd.DataFrame.from_dict(map(dict,out_put['tables']))
                #st.write(df)
                
                if int(out_put['length']) != 0:
                    st.markdown(get_csv_download_link(out_put["tables"]), unsafe_allow_html =True)
                    
                    st.markdown(get_text_download_link(out_put['text']),unsafe_allow_html = True)
                    
                    
                else:
                    st.markdown(get_text_download_link(out_put['text']),unsafe_allow_html = True)
                    
                    
elif choice == "Q/A System":
    st.title("Q/A System")

elif choice == "Recommendation System":
    st.title("Recommendation System")

elif choice == "Topic Modelling":
    st.title("Topic Modelling")
    


# In[ ]:






2021-07-23 20:12:18.952 
  command:

    streamlit run C:\Users\mukes\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py [ARGUMENTS]
